doc template by : http://www.axainsurance.com/home/policy-wording/policywording_153.pdf

# Load PDF

In [5]:
from langchain.document_loaders import PyMuPDFLoader

local_path = "sample_policy_doc_AU1234.pdf"

if local_path:
    loader = PyMuPDFLoader(local_path)
    data = loader.load()
else:
    print("Upload a PDF file for processing.")

# Converting content into dense vector embeddings 

In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma 

In [7]:
# Split and chunk the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)
chroma_persist_directory = "chroma_db"


# Add the chunks to vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name="police-rag",
    persist_directory=chroma_persist_directory,
)

OllamaEmbeddings: 100%|██████████| 58/58 [00:05<00:00, 11.60it/s]


In [8]:
# Save the FAISS index for future use
vector_db.persist()

/var/folders/4s/60c4fc9n1f92ww4_fr6ds79r0000gn/T/ipykernel_15673/1784304188.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


# Retrieval + Generation of Response

In [9]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

local_llm = "llama3.1"
llm = ChatOllama(model=local_llm)

# Define the query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI assistant. Please provide answers that are relevant to the customer's question, retrieve relevant documents from a vector database, and ensure the text is clean with proper punctuation and precise answers. 
    Original question: {question}"""
)

retriever = MultiQueryRetriever.from_llm(vector_db.as_retriever(), llm, prompt=QUERY_PROMPT)

/var/folders/4s/60c4fc9n1f92ww4_fr6ds79r0000gn/T/ipykernel_15673/2787540018.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_llm)


# Adding an Agent

In [10]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import tool


@tool("ask_pdf")
def ask_pdf_tool(question: str) -> str:
    """A tool to query the PDF content for relevant answers."""
    # Perform the retrieval process directly here
    results = retriever.get_relevant_documents(question)
    return " ".join([result.page_content for result in results])

tools = [
    Tool(name="PDF Query Tool", func=ask_pdf_tool, description="Tool to query PDF data")
]

# Initialize the agent with tools and language model
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# CHATBOT

In [11]:
import logging

# Disable the debug and internal logging from LangChain
logging.basicConfig(level=logging.CRITICAL)

In [12]:
# Chatbot with history
history = []

def chatbot(query: str):
    # Add the user question to the history
    history.append(f"User: {query}")
    
    # Get the response from the agent
    # response = agent.run(query)
    response = ask_pdf_tool(query)
    
    # Add the response to the history
    history.append(f"Agent: {response}")
    
    # Return the response with the history
    return "\n".join(history)

# Example query
question = "Will you pay for accidental damage or loss of contents while they are in the home?"
response = chatbot(question)

# Display the response
print(response)

/var/folders/4s/60c4fc9n1f92ww4_fr6ds79r0000gn/T/ipykernel_15673/2643287435.py:10: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = ask_pdf_tool(query)
/var/folders/4s/60c4fc9n1f92ww4_fr6ds79r0000gn/T/ipykernel_15673/1076426864.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(question)
OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 56.56it/s]

User: Will you pay for accidental damage or loss of contents while they are in the home?
Agent: ADH 15.10a 
9 
Contents Insurance 
 
What your policy covers:  
What your policy does not cover: 
We will pay you up to the maximum contents cover 
limit for any one claim under contents and for the 
cover and causes detailed in this section. 
 
Contents cover limit – please refer to your schedule 
 
There is a separate limit for the following, these can 
be found in your schedule: 
 
• 
Valuables – see definition. 
• 
Single valuables limit  
 
Any other specific limits are identified below. 
 
 
1. Excess – shown in schedule. 
 
2. Items listed under the general exclusions – see 
separate document – Essential Information. 
 
3. The exclusions listed below which relate to the 
corresponding cover identified in the first 
column. 
 
4. Motorised vehicle or craft and caravans as 
well as parts, accessories, tools, fitted radios, 
cassette players and compact disc players and 
satellite naviga

In [13]:
# Example query
question = "Will you pay for accidental damage or loss of contents while they are in the home?"
response = chatbot(question)

# Display the response
print(response)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 52.54it/s]

User: Will you pay for accidental damage or loss of contents while they are in the home?
Agent: ADH 15.10a 
9 
Contents Insurance 
 
What your policy covers:  
What your policy does not cover: 
We will pay you up to the maximum contents cover 
limit for any one claim under contents and for the 
cover and causes detailed in this section. 
 
Contents cover limit – please refer to your schedule 
 
There is a separate limit for the following, these can 
be found in your schedule: 
 
• 
Valuables – see definition. 
• 
Single valuables limit  
 
Any other specific limits are identified below. 
 
 
1. Excess – shown in schedule. 
 
2. Items listed under the general exclusions – see 
separate document – Essential Information. 
 
3. The exclusions listed below which relate to the 
corresponding cover identified in the first 
column. 
 
4. Motorised vehicle or craft and caravans as 
well as parts, accessories, tools, fitted radios, 
cassette players and compact disc players and 
satellite naviga